In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from src.wimbd_ import BasePaths as PATHS
from src.wimbd_ import DataConfigs as CONFIG
from src.wimbd_ import post_filter, filter_percentile, load_results
from src.wimbd_ import ProcessTriviaQA as ptqa
from src.utils import remove_nested_lists
from src.analysis import AnalyzeNgrams as an
from wimbd_process_results import softmax

from datetime import datetime
pd.set_option('display.max_columns', 10)

# Generate a timestamp
timestamp = datetime.now().strftime("%Y-%m-%d-%H:%M")

[nltk_data] Downloading package punkt to
[nltk_data]     /local/home/antonis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /local/home/antonis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[dynet] random seed: 1234
[dynet] allocating memory: 32MB
[dynet] memory allocation done.


ModuleNotFoundError: No module named 'wimbd_process_results'

In [ ]:
task_coverage_pth = "./results/n-grams/triviaqa/pile/exp_3/validation-set/n_samples_None_fkeyFalse_rkeyFalse_fstopTrue_onlyalphaFalse/3/common/task-p-coverage.pkl"
task_coverage = pd.read_pickle(task_coverage_pth)

In [ ]:
all_models = ['pythia-12b', 'pythia-6.9b', 'pythia-2.8b', 'pythia-1.4b', 'pythia-410m', 'pythia-160m', 'pythia-70m', 'pythia-31m', 'pythia-14m']
large_models = ['pythia-12b', 'pythia-6.9b', 'pythia-2.8b', 'pythia-1.4b']
small_models = ['pythia-410m', 'pythia-160m', 'pythia-70m', 'pythia-31m', 'pythia-14m']

N_GRAMS = 5
ALIGN_THRESH = 85
# BASE_DIR = f"./results/n-grams/trivia_qa/pile/exp_3/test-set/n_samples_None_fkeyFalse_rkeyFalse_fstopTrue_onlyalphaFalse/"
# BASE_DIR = f"./results/n-grams/triviaqa/pile/exp_3/validation-set/n_samples_None_fkeyFalse_rkeyFalse_fstopTrue_onlyalphaFalse/"
BASE_DIR = f"./results/n-grams/triviaqa/pile/exp_3/validation-set/n_samples_None_fkeyFalse_rkeyFalse_fstopFalse_onlyalphaFalse"
DATASET = "examples_dfs_4-shot_common_models.pkl"
TASKS = "triviaqa"
OMMIT_TASKS = False
TASKS_OMMIT = ["formal_logic"]
POST_FILTER = True

# BASE_DIR = PATHS.base_ngram_paths[TASKS]['base_path']
BASE_PATH = os.path.join(BASE_DIR, f"{N_GRAMS}")
METHOD = "0-shot_common"
BASE_PATH_COMMON = os.path.join(BASE_PATH, "common")
BASE_PATH_ALL = os.path.join(BASE_PATH, "all")
# FIG_DIR = os.path.join("/share/edc/home/antonis/LLM-Incidental-Supervision/incidental-supervision/figures", DATASET, timestamp)
# if not os.path.exists(FIG_DIR):
#     os.makedirs(FIG_DIR)

# RESULTS_PATH = os.path.join(BASE_DIR, str(N_GRAMS), 'common', DATASET)
RESULTS_PATH = os.path.join(BASE_DIR, str(N_GRAMS), DATASET)
FIGS_PTH = "./figures/trivia_qa"

print(f"BASE PATH: {BASE_PATH}")

VIEW_COLS = ['Q', 'A', 'question', 'value', 'align_score', 'em', 'answer']

In [ ]:
"./results/n-grams/triviaqa/pile/exp_3/validation-set/n_samples_None_fkeyFalse_rkeyFalse_fstopTrue_onlyalphaFalse/3/examples_dfs_4-shot_common_models.pkl"

In [ ]:
RESULTS_PATH == "./results/n-grams/triviaqa/pile/exp_3/validation-set/n_samples_None_fkeyFalse_rkeyFalse_fstopTrue_onlyalphaFalse/3/examples_dfs_4-shot_common_models.pkl"

In [ ]:
RESULTS_PATH

In [ ]:
# filter outliers
pickle_file = pd.read_pickle(RESULTS_PATH)
# df = pd.DataFrame(pickle_file).T.sort_values("value", ascending=False)


# filter
# open
examples_models_pth = os.path.join(BASE_PATH, f"examples_dfs_{METHOD}_post-processed.pkl")
if os.path.exists(examples_models_pth):
    with open(examples_models_pth, 'rb') as f:
        examples_models = pickle.load(f)
else:
    examples_models = ptqa.process_model_examples(pickle_file, p=0.99999)
examples_model = examples_models['pythia-12b']
colors = sns.color_palette('hls', len(examples_models))

In [ ]:
examples_model.head(20)

In [ ]:
# # save
# with open(os.path.join(BASE_PATH, f"examples_dfs_{METHOD}_post-processed.pkl"), 'wb') as f:
#     pickle.dump(examples_models, f)

In [ ]:
examples_models_ = {model: data[data['align_score'] > ALIGN_THRESH] for model, data in examples_models.items()}

In [ ]:
# examples_model_small = examples_model.iloc[0:3]
# examples_model_small_filtered, indices_kept = ptqa.calculate_alignment_scores(examples_model_small, 75, 1)

In [ ]:
# display_cols = ['Q', 'A', 'value', 'example']
# pd.set_option('display.max_rows', 1000)
# pd.set_option('display.max_colwidth', 2)  # Set to None to display full content
# examples_model[display_cols].iloc[200:400]

In [ ]:
from src.analysis import AnalyzeNgrams as an

# coverage_path = os.path.join(BASE_PATH_COMMON, "task-coverage.pkl")
# task_cov = an.calculate_average_task_coverage(BASE_PATH_ALL, [TASKS], [N_GRAMS])

In [ ]:
# # Assuming 'value' is the column with values to bin and we want 5 bins between 1 and 10000
# examples_model = an.create_bins(examples_model, 'value', 1, 10000, 4)

# example_samples_model = ptqa.get_random_samples_by_interval(examples_model, 100)

# ptqa.plot_avg_em_by_log_interval(example_samples_model, log_axis=True)
# ptqa.plot_avg_em_by_log_interval(examples_model, log_axis=True)
# # plot_avg_em_by_log_interval(example_samples_model, log_axis=False)

In [ ]:
len(examples_models_['pythia-12b'])

In [ ]:
# Create a figure and axis
fig, ax = plt.subplots(figsize=(12, 8))

color_palette = sns.color_palette('husl', len(examples_models))
is_log = True
plot_var = 'value'
savefile = os.path.join(FIGS_PTH, f"avg_em_by_log_interval_all_log{is_log}_{plot_var}_aligned{ALIGN_THRESH}.png")
for i, (model_name, data) in enumerate(examples_models_.items()):
    data = an.create_bins(data, plot_var, 0, 10000, 8, is_log)
    # data = ptqa.get_random_samples_by_interval(data, 85)
    ax = ptqa.plot_variable_by_interval(data, log_axis=is_log,
                                 x_column='interval', y_column='em',
                                 save_pth=savefile, ax=ax, 
                                 color=color_palette[i], label=model_name,
                                 title=f'Performance vs. Ngrams for Different Models, {TASKS}, N={N_GRAMS}',
                                 plot_std=True)

In [ ]:
examples_model = examples_models['pythia-12b']
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 500)
# Corrected code
pd.set_option('display.max_colwidth', 1)
examples_model[VIEW_COLS].head(5)

In [ ]:
question = "What is the medical term for high blood pressure?"
df_q = examples_model[examples_model['question'] == question]
df_q['Q']

In [ ]:
from src.utils import normalize_string

# Apply the function to each row of the DataFrame
# examples_model[['q_coverage', 'a_coverage']] = examples_model.apply(calculate_qa_coverage, axis=1)

coverage_df = an.calculate_qa_coverage(examples_model)

In [ ]:
coverage_df.head(1)

In [ ]:
coverage_df['q_coverage'].unique()

In [ ]:
coverage_df[coverage_df['q_coverage'] < 0]

In [ ]:
is_log = False
plot_var = 'q_coverage'
x_label = 'Question Coverage (p)'

examples_model_cov_int = an.create_bins(coverage_df, plot_var, 
                                        -0.1, 1.0, 5, 
                                        is_log=is_log)

ptqa.plot_variable_by_interval(examples_model_cov_int, log_axis=is_log,
                                x_column='interval', y_column='em',
                                save_pth=savefile, 
                                label='pythia-12b',
                                title=f'Performance vs. {plot_var} for Different Models N={N_GRAMS}',
                                x_label=x_label, 
                                plot_std=True)

In [ ]:
def calculate_coverage(col_1, col_2):
    

In [ ]:
def calculate_coverage()

In [ ]:
examples_model.groupby('log_interval').count()